# Wrangling DataFrames with Pandas    
## Author: Erika Duan    

![](../02_figures/04_manipulating-pandas-header.jpg)

# Dataset background  

We will be using open government data to familiarise ourselves with Pandas DataFrame manipulations. The dataset we will use describes the [index of disadvantage by schools in South Australia](https://data.gov.au/dataset/ds-sa-4ab8a539-eab5-48e2-8d88-a599d6114126/details?q=school), with a focus on [2020 data](https://data.gov.au/dataset/ds-sa-4ab8a539-eab5-48e2-8d88-a599d6114126/distribution/dist-sa-86d35483-feff-42b5-ac05-ad3186ac39de/details?q=school).     

+ The index is a socio-economic index, used by the South Australian Department for Education to allocate resources to schools to address educational disadvantage related to socio-economic status.  
+ The most disadvantaged schools have an index of 1.    
+ The least disadvantaged have an index of 7.   
+ Data is currently available for 2017, 2018 and 2020.  

# Loading data     

We can load data in two different ways:  

1. Locally using the `os` library and the Pandas `read_csv` function.  
2. Directly through the data.gov.au [CKAN data API](https://docs.ckan.org/en/latest/maintaining/datastore.html) as documented [here]().    

In [28]:
#-----method 1: load csv file locally-----  
import pathlib
import pandas as pd     

working_dir = pathlib.Path.cwd() # get current working directory  
data_path = working_dir.parent / '01_raw_data' / '04_index-of-disadvantage-by-school-2020.csv'

raw_data = pd.read_csv(data_path)  
raw_data.head()

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743
3,4,997,Adelaide East Education Centre,Paradise,5075,Special Education,6,-34.873042,138.672971
4,5,768,Adelaide High School,Adelaide,5000,Secondary Education,5,-34.925733,138.586678


In [53]:
#-----method 2: access data as JSON file through CKAN data API-----  
import pathlib
import numpy as np
import pandas as pd
import requests
import json

url = "https://data.sa.gov.au/data/api/3/action/datastore_search?resource_id=86d35483-feff-42b5-ac05-ad3186ac39de&limit=512"
json_content = requests.get(url).json() # returns a dictionary object 

content = json.dumps(json_content, indent = 4, sort_keys = True) # JSON encoder returns a JSON file format/ string object 

df = pd.read_json(content)
df = pd.json_normalize(df['result']['records']) # normalize semi-structured JSON data into a flat table
df.head()

,ID,Index_of_Educational_Disadvantage,Latitude,Longitude,Post_Code,School_Name,School_Number,Suburb,Type_of_Schooling,_id
0,1,6,-35.078139,138.595142,5159,Aberfoyle Hub R-7 School,536,Aberfoyle Park,Primary Education,1
1,2,6,-35.074007,138.594181,5159,Aberfoyle Park High School,1673,Aberfoyle Park,Secondary Education,2
2,3,7,-34.917055,138.606743,5000,Adelaide Botanic High School,1919,Adelaide,Secondary Education,3
3,4,6,-34.873042,138.672971,5075,Adelaide East Education Centre,997,Paradise,Special Education,4
4,5,5,-34.925733,138.586678,5000,Adelaide High School,768,Adelaide,Secondary Education,5


**Note:** An introduction to the JSON file format can be found [here](https://towardsdatascience.com/an-introduction-to-json-c9acb464f43e).  

# Examining DataFrame structure  

We can examine the properties of our DataFrame using the following functions and methods:  
+ `df.shape()` to return the DataFrame dimensions.     
+ `df.info()` to return a summary of all DataFrame columns, missing values and column object types.    
+ `df.columns` to return an index of all column names.  
+ `df.describe()` to return a simple statistical summary of each column's values.  

**Note:** You can also configure Jupyter notebook to display all columns with `pd.set_option("display.max.columns", None)`. 

In [45]:
#-----example 1.1-----  
raw_data.shape

(512, 9)

In [44]:
#-----example 1.2----- 
print("The raw data has {} rows and {} columns.".format(raw_data.shape[0], raw_data.shape[1]))

The raw data has 512 rows and 9 columns.


In [46]:
#-----example 2-----  
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID                                 512 non-null    int64  
 1   School_Number                      512 non-null    int64  
 2   School_Name                        512 non-null    object 
 3   Suburb                             512 non-null    object 
 4   Post_Code                          512 non-null    int64  
 5   Type_of_Schooling                  512 non-null    object 
 6   Index_of_Educational_Disadvantage  512 non-null    int64  
 7   Latitude                           512 non-null    float64
 8   Longitude                          512 non-null    float64
dtypes: float64(2), int64(4), object(3)
memory usage: 36.1+ KB


In [51]:
#-----example 3-----  
raw_data.columns

Index(['ID', 'School_Number', 'School_Name', 'Suburb', 'Post_Code',
       'Type_of_Schooling', 'Index_of_Educational_Disadvantage', 'Latitude',
       'Longitude'],
      dtype='object')

In [52]:
#-----example 4-----  
raw_data.describe() # only operates on columns containing integers or floats  

,ID,School_Number,Post_Code,Index_of_Educational_Disadvantage,Latitude,Longitude
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000
mean,256.500000,771.277344,5167.619141,4.304688,-34.592120,138.502559
std,147.945936,451.007245,608.153567,1.868719,1.633955,1.529273
min,1.000000,104.000000,872.000000,1.000000,-38.002525,129.172118
25%,128.750000,388.500000,5085.750000,3.000000,-35.029207,138.505453
50%,256.500000,754.500000,5163.000000,5.000000,-34.856216,138.621977
75%,384.250000,1017.250000,5351.000000,6.000000,-34.440930,138.768058
max,512.000000,1919.000000,5734.000000,7.000000,-26.151709,140.956465


# Subsetting DataFrame rows and columns   

## Subsetting DataFrame rows  

As [previously covered](https://github.com/erikaduan/Introduction-to-Python/blob/master/03_notebooks/03_Python-data-structures.ipynb), subsetting DataFrame rows can be approached in two different ways:   

+ Using `loc` i.e. via the explicit labels-based index.  
+ Using `iloc` i.e. via the implicit 0-based index.  

In [71]:
#-----example 1----- 
raw_data.iloc[0:2+1] # subset the first 3 rows    

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [63]:
#-----example 2-----  
raw_data.iloc[0:2+1, 1] # returns the first 3 rows from the second column    

0     536
1    1673
2    1919
Name: School_Number, dtype: int64

In [73]:
#-----example 4-----  
raw_data.loc[0:2+1, 'School_Number'] # returns the first 3 rows from column 'School_Number'

# note that raw_data.loc["subject"] produces an error
# whereas raw_data.loc[:, "subject"] does not   

0     536
1    1673
2    1919
3     997
Name: School_Number, dtype: int64

## Filtering by DataFrame rows  

## Subsetting DataFrame columns

In [39]:
students[students["id"] == 2] # filtering by rows  

,id,subject,hours_per_day
1,2,science,2.0


In [43]:
student_list = [1, 2, 5]

students[students["id"].isin(student_list)] # use the isin() method for a list of values  

,id,subject,hours_per_day
0,1,maths,2.0
1,2,science,2.0
4,5,music,3.0


In [69]:
#-----transform columns i.e.------
students[["hours_per_week"]] =  students.apply(lambda row: row[["hours_per_day"]] * 7, axis = 1)  

students

,id,subject,hours_per_day,hours_per_week
0,1,maths,2.0,14.0
1,2,science,2.0,14.0
2,3,english,1.5,10.5
3,4,science,1.0,7.0
4,5,music,3.0,21.0


In [60]:
a = [1, 2, 3]
print(a) 

b = [4, 5, 6] 
print(b)

a[0]

[1, 2, 3]
[4, 5, 6]


1

In [35]:
# subset DataFrame columns
students[["id", "subject"]]  

print(type(students["id"])) # series
print(type(students[["id"]])) # DataFrame

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
